In [1]:
!pip install transformers

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from numpy import newaxis
import math
import os
import pandas as pd
import torch.nn as nn
from scipy.stats import chi2
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
# configuration for training, you should modify these values to get the best performance
config = {
    "num_labels": 7,
    "hidden_dropout_prob": 0.15,
    "hidden_size": 768,
    "max_length": 512,
}

training_parameters = {
    "batch_size": 16,
    "epochs": 10,
    "output_folder": "/kaggle/working",
    "output_file": "model.bin",
    "learning_rate": 2e-5,
    "print_after_steps": 100,
    "save_steps": 5000,

}

In [5]:
class ReviewDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.tokenizer = AutoTokenizer.from_pretrained('jackaduma/SecBERT')

    def __getitem__(self, index):
        review = self.df.iloc[index]["text"]
        attack = self.df.iloc[index]["label"]
        attack_dict = {'000 - Normal': 0,
          '126 - Path Traversal': 1,
          '242 - Code Injection': 2,
          '153 - Input Data Manipulation': 3,
          '310 - Scanning for Vulnerable Software': 4,
          '194 - Fake the Source of Data': 5,
          '34 - HTTP Response Splitting': 6}
        label = attack_dict[attack]
        encoded_input = self.tokenizer.encode_plus(
                review,
                add_special_tokens=True,
                max_length = 512,
                padding="max_length",
                return_overflowing_tokens=True,
                truncation = True,
            )
        if "num_truncated_tokens" in encoded_input and encoded_input["num_truncated_tokens"] > 0:
            # print("Attention! you are cropping tokens")
            pass

        input_ids = encoded_input["input_ids"]
        attention_mask = encoded_input["attention_mask"] if "attention_mask" in encoded_input else None

        token_type_ids = encoded_input["token_type_ids"] if "token_type_ids" in encoded_input else None



        data_input = {
            "input_ids": torch.tensor(input_ids),
            "attention_mask": torch.tensor(attention_mask),
            "token_type_ids": torch.tensor(token_type_ids),
            "label": torch.tensor(label),
        }

        return data_input["input_ids"], data_input["attention_mask"], data_input["token_type_ids"], data_input["label"]



    def __len__(self):
        return self.df.shape[0]

In [6]:
from typing import Optional, Sequence

class GaussianKernel(nn.Module):
    r"""Gaussian Kernel Matrix
    Gaussian Kernel k is defined by
    .. math::
        k(x_1, x_2) = \exp \left( - \dfrac{\| x_1 - x_2 \|^2}{2\sigma^2} \right)
    where :math:`x_1, x_2 \in R^d` are 1-d tensors.
    Gaussian Kernel Matrix K is defined on input group :math:`X=(x_1, x_2, ..., x_m),`
    .. math::
        K(X)_{i,j} = k(x_i, x_j)
    Also by default, during training this layer keeps running estimates of the
    mean of L2 distances, which are then used to set hyperparameter  :math:`\sigma`.
    Mathematically, the estimation is :math:`\sigma^2 = \dfrac{\alpha}{n^2}\sum_{i,j} \| x_i - x_j \|^2`.
    If :attr:`track_running_stats` is set to ``False``, this layer then does not
    keep running estimates, and use a fixed :math:`\sigma` instead.
    Args:
        sigma (float, optional): bandwidth :math:`\sigma`. Default: None
        track_running_stats (bool, optional): If ``True``, this module tracks the running mean of :math:`\sigma^2`.
          Otherwise, it won't track such statistics and always uses fix :math:`\sigma^2`. Default: ``True``
        alpha (float, optional): :math:`\alpha` which decides the magnitude of :math:`\sigma^2` when track_running_stats is set to ``True``
    Inputs:
        - X (tensor): input group :math:`X`
    Shape:
        - Inputs: :math:`(minibatch, F)` where F means the dimension of input features.
        - Outputs: :math:`(minibatch, minibatch)`
    """

    def __init__(self, sigma: Optional[float] = None, track_running_stats: Optional[bool] = True,
                 alpha: Optional[float] = 1.):
        super(GaussianKernel, self).__init__()
        assert track_running_stats or sigma is not None
        self.sigma_square = torch.tensor(sigma * sigma) if sigma is not None else None
        self.track_running_stats = track_running_stats
        self.alpha = alpha

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        l2_distance_square = ((X.unsqueeze(0) - X.unsqueeze(1)) ** 2).sum(2)

        if self.track_running_stats:
            self.sigma_square = self.alpha * torch.mean(l2_distance_square.detach())

        return torch.exp(-l2_distance_square / (2 * self.sigma_square))

class MultipleKernelMaximumMeanDiscrepancy(nn.Module):
    r"""The Multiple Kernel Maximum Mean Discrepancy (MK-MMD) used in
    `Learning Transferable Features with Deep Adaptation Networks (ICML 2015) `_
    Given source domain :math:`\mathcal{D}_s` of :math:`n_s` labeled points and target domain :math:`\mathcal{D}_t`
    of :math:`n_t` unlabeled points drawn i.i.d. from P and Q respectively, the deep networks will generate
    activations as :math:`\{z_i^s\}_{i=1}^{n_s}` and :math:`\{z_i^t\}_{i=1}^{n_t}`.
    The MK-MMD :math:`D_k (P, Q)` between probability distributions P and Q is defined as
    .. math::
        D_k(P, Q) \triangleq \| E_p [\phi(z^s)] - E_q [\phi(z^t)] \|^2_{\mathcal{H}_k},
    :math:`k` is a kernel function in the function space
    .. math::
        \mathcal{K} \triangleq \{ k=\sum_{u=1}^{m}\beta_{u} k_{u} \}
    where :math:`k_{u}` is a single kernel.
    Using kernel trick, MK-MMD can be computed as
    .. math::
        \hat{D}_k(P, Q) &=
        \dfrac{1}{n_s^2} \sum_{i=1}^{n_s}\sum_{j=1}^{n_s} k(z_i^{s}, z_j^{s})\
        &+ \dfrac{1}{n_t^2} \sum_{i=1}^{n_t}\sum_{j=1}^{n_t} k(z_i^{t}, z_j^{t})\
        &- \dfrac{2}{n_s n_t} \sum_{i=1}^{n_s}\sum_{j=1}^{n_t} k(z_i^{s}, z_j^{t}).\
    Args:
        kernels (tuple(torch.nn.Module)): kernel functions.
        linear (bool): whether use the linear version of DAN. Default: False
    Inputs:
        - z_s (tensor): activations from the source domain, :math:`z^s`
        - z_t (tensor): activations from the target domain, :math:`z^t`
    Shape:
        - Inputs: :math:`(minibatch, *)`  where * means any dimension
        - Outputs: scalar
    .. note::
        Activations :math:`z^{s}` and :math:`z^{t}` must have the same shape.
    .. note::
        The kernel values will add up when there are multiple kernels.
    Examples::
        >>> from tllib.modules.kernels import GaussianKernel
        >>> feature_dim = 1024
        >>> batch_size = 10
        >>> kernels = (GaussianKernel(alpha=0.5), GaussianKernel(alpha=1.), GaussianKernel(alpha=2.))
        >>> loss = MultipleKernelMaximumMeanDiscrepancy(kernels)
        >>> # features from source domain and target domain
        >>> z_s, z_t = torch.randn(batch_size, feature_dim), torch.randn(batch_size, feature_dim)
        >>> output = loss(z_s, z_t)
    """

    def __init__(self, kernels: Sequence[nn.Module], linear: Optional[bool] = False):
        super(MultipleKernelMaximumMeanDiscrepancy, self).__init__()
        self.kernels = kernels
        self.index_matrix = None
        self.linear = linear

    def forward(self, z_s: torch.Tensor, z_t: torch.Tensor) -> torch.Tensor:
        features = torch.cat([z_s, z_t], dim=0)
        batch_size = int(z_s.size(0))
        self.index_matrix = _update_index_matrix(batch_size, self.index_matrix, self.linear).to(z_s.device)


        kernel_matrix = sum([kernel(features) for kernel in self.kernels])  # Add up the matrix of each kernel
        # Add 2 / (n-1) to make up for the value on the diagonal
        # to ensure loss is positive in the non-linear version
        loss = (kernel_matrix * self.index_matrix).sum() + 2. / float(batch_size - 1)

        return loss


def _update_index_matrix(batch_size: int, index_matrix: Optional[torch.Tensor] = None,
                         linear: Optional[bool] = True) -> torch.Tensor:
    r"""
    Update the `index_matrix` which convert `kernel_matrix` to loss.
    If `index_matrix` is a tensor with shape (2 x batch_size, 2 x batch_size), then return `index_matrix`.
    Else return a new tensor with shape (2 x batch_size, 2 x batch_size).
    """
    if index_matrix is None or index_matrix.size(0) != batch_size * 2:
        index_matrix = torch.zeros(2 * batch_size, 2 * batch_size)
        if linear:
            for i in range(batch_size):
                s1, s2 = i, (i + 1) % batch_size
                t1, t2 = s1 + batch_size, s2 + batch_size
                index_matrix[s1, s2] = 1. / float(batch_size)
                index_matrix[t1, t2] = 1. / float(batch_size)
                index_matrix[s1, t2] = -1. / float(batch_size)
                index_matrix[s2, t1] = -1. / float(batch_size)
        else:
            for i in range(batch_size):
                for j in range(batch_size):
                    if i != j:
                        index_matrix[i][j] = 1. / float(batch_size * (batch_size - 1))
                        index_matrix[i + batch_size][j + batch_size] = 1. / float(batch_size * (batch_size - 1))
            for i in range(batch_size):
                for j in range(batch_size):
                    index_matrix[i][j + batch_size] = -1. / float(batch_size * batch_size)
                    index_matrix[i + batch_size][j] = -1. / float(batch_size * batch_size)
    return index_matrix

In [7]:
df_train = pd.read_csv('/kaggle/input/code-injection/dataset_capec_combine.csv')
df_train.head()

,text,label
0,GET /blog/index.php/2020/04/04/voluptatum-repr...,000 - Normal
1,GET /blog/xmlrpc.php?rsd,000 - Normal
2,GET /blog/index.php/2020/04/04/nihil-tenetur-e...,000 - Normal
3,GET /blog/index.php/2020/04/04/explicabo-qui-f...,000 - Normal
4,GET /blog/index.php/2020/04/04/explicabo-qui-f...,000 - Normal


In [8]:
# Optional (not effect very much)
# for word tokenizer instead of character tokenizer
df_train['text'] = df_train['text'].str.replace('/',' ')
df_train.head()

,text,label
0,GET blog index.php 2020 04 04 voluptatum-repr...,000 - Normal
1,GET blog xmlrpc.php?rsd,000 - Normal
2,GET blog index.php 2020 04 04 nihil-tenetur-e...,000 - Normal
3,GET blog index.php 2020 04 04 explicabo-qui-f...,000 - Normal
4,GET blog index.php 2020 04 04 explicabo-qui-f...,000 - Normal


In [9]:
from sklearn.model_selection import train_test_split
## prepare for training
df_train = df_train[0:len(df_train)//training_parameters['batch_size']*training_parameters['batch_size']]
source_dataset = ReviewDataset(df_train)
source_dataloader = DataLoader(dataset = source_dataset, batch_size = training_parameters["batch_size"], shuffle = True, num_workers = 2)

In [10]:
df_transfer = pd.read_csv('/kaggle/input/code-injection/dataset_capec_transfer.csv')
df_transfer.head()

,text,label
0,POST /vendor/phpunit/phpunit/src/Util/PHP/eval...,153 - Input Data Manipulation
1,POST /cgi-bin/ViewLog.asp remote_submit_Flag=...,153 - Input Data Manipulation
2,GET /.svn/wc.db,153 - Input Data Manipulation
3,GET /blog/.svn/wc.db,153 - Input Data Manipulation
4,GET /blog/index.php/my-account/.svn/wc.db,153 - Input Data Manipulation


In [11]:
# Optional (not effect very much)
# for word tokenizer instead of character tokenizer
df_transfer['text'] = df_transfer['text'].str.replace('/',' ')
df_transfer = df_transfer[0:len(df_transfer)//training_parameters['batch_size']*training_parameters['batch_size']]

In [12]:
target_dataset = ReviewDataset(df_transfer)
target_dataloader = DataLoader(dataset = target_dataset, batch_size = training_parameters["batch_size"], shuffle = True, num_workers = 2)

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim

class DomainAdaptationModel(nn.Module):
    def __init__(self):
        super(DomainAdaptationModel, self).__init__()

        num_labels = config["num_labels"]
        self.bert = AutoModel.from_pretrained('jackaduma/SecBERT') # model that we will use
        self.dropout = nn.Dropout(config["hidden_dropout_prob"])

        self.prj = nn.Linear(config["hidden_size"], config["hidden_size"]//2);

        self.attack_classifier = nn.Sequential(
            nn.Linear(config["hidden_size"]//2, num_labels),
            nn.LogSoftmax(dim=1),
        )


#       Freeze bert layer
        modules = [self.bert.embeddings, self.bert.encoder.layer[:2]] #Replace value by what you want
        for module in modules:
            for param in module.parameters():
                param.requires_grad = False


    def forward(
          self,
          input_ids=None,
          attention_mask=None,
          token_type_ids=None,
          labels=None,
#           grl_lambda = 1.0,
          ):

        outputs = self.bert(
                input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
            )

#         pooled_output = outputs[1] # For bert-base-uncase
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)

        pooled_output_prj = self.prj(pooled_output)

        attack_pred = self.attack_classifier(pooled_output_prj)

        return attack_pred.to(device), pooled_output_prj

In [14]:
def compute_accuracy(logits, labels):
    predicted_labels_dict = {
      0: 0,
      1: 0,
      2: 0,
      3: 0,
      4: 0,
      5: 0,
      6: 0,
    }

    predicted_label = logits.max(dim = 1)[1]

    for pred in predicted_label:
        # print(pred.item())
        predicted_labels_dict[pred.item()] += 1
    acc = (predicted_label == labels).float().mean()

    return acc, predicted_labels_dict

In [15]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report,accuracy_score, f1_score

def evaluate(model, dataset = "target", percentage = 80):
    with torch.no_grad():
        predicted_labels_dict = {
          0: 0,
          1: 0,
          2: 0,
          3: 0,
          4: 0,
          5: 0,
          6: 0,
        }
        model.eval()
        dev_df = pd.read_csv("/kaggle/input/code-injection/dataset_capec_" + dataset + ".csv")
        data_size = dev_df.shape[0]
        selected_for_evaluation = int(data_size*percentage/100)
        dev_df = dev_df.head(selected_for_evaluation)
        dataset = ReviewDataset(dev_df)
        dataloader = DataLoader(dataset = dataset, batch_size = training_parameters["batch_size"], shuffle = True, num_workers = 2)

        true_labels = list()
        predicted_label = list()
        for input_ids, attention_mask, token_type_ids, labels in dataloader:
            inputs = {
                "input_ids": input_ids.squeeze(axis=1),
                "attention_mask": attention_mask.squeeze(axis=1),
                "token_type_ids" : token_type_ids.squeeze(axis=1),
                "labels": labels,
            }
            for k, v in inputs.items():
                inputs[k] = v.to(device)
            attack_pred, _ = model(**inputs)
            true_labels.extend(attack_pred.max(dim = 1)[1].cpu().numpy())
            predicted_label.extend(inputs['labels'].cpu().numpy())
            _, predicted_labels = compute_accuracy(attack_pred, inputs["labels"])

            for i in range(7):
              predicted_labels_dict[i] += predicted_labels[i]

        score = f1_score(true_labels,predicted_label,average="macro")
        precision = precision_score(true_labels, predicted_label,average="macro")
        recall = recall_score(true_labels, predicted_label,average="macro")
        report = classification_report(true_labels,predicted_label,digits=4)
        acc= accuracy_score(true_labels, predicted_label)
        #classifaction_report_csv(report,precision,recall,score,0)
        print ('\n clasification report:\n', report)
        print ('F1 score:', score)
        print ('Recall:', recall)
        print ('Precision:', precision)
        print ('Acc:', acc)
        print('Confusion Matrix: \n',confusion_matrix(true_labels, predicted_label))
        print(predicted_labels_dict)

In [16]:
lr = training_parameters["learning_rate"]
n_epochs = training_parameters["epochs"]

In [17]:
lr = training_parameters["learning_rate"]
n_epochs = training_parameters["epochs"]

model = DomainAdaptationModel()
model.to(device)

optimizer = optim.Adam(model.parameters(), lr)

loss_fn_attack_classifier = torch.nn.NLLLoss()
# loss_fn_domain_classifier = torch.nn.NLLLoss()
mkmmd_loss = MultipleKernelMaximumMeanDiscrepancy(
        kernels=[GaussianKernel(alpha=2 ** k) for k in range(-3, 2)],
        linear=True
    )


'''
In one training step we will update the model using both the source labeled data and target unlabeled data
We will run it till the batches last for any of these datasets

In our case target dataset has more data. Hence, we will leverage the entire source dataset for training

If we use the same approach in a case where the source dataset has more data then the target dataset then we will
under-utilize the labeled source dataset. In such a scenario it is better to reload the target dataset when it finishes
This will ensure that we are utilizing the entire source dataset to train our model.
'''

max_batches = min(len(source_dataloader), len(target_dataloader))
best_acc = 0.
for i in range(3):
    for epoch_idx in range(10):
        source_iterator = iter(source_dataloader)
        target_iterator = iter(target_dataloader)
        loss_after_epoch = 0.
        for batch_idx in range(max_batches):

            p = float(batch_idx + epoch_idx * max_batches) / (training_parameters["epochs"] * max_batches)
            grl_lambda = 2. / (1. + np.exp(-10 * p)) - 1
            grl_lambda = torch.tensor(grl_lambda)

            model.train()
            # if(batch_idx%training_parameters["print_after_steps"] == 0 ):
            #     print("Training Step:", batch_idx)

            optimizer.zero_grad()

            # Souce dataset training update
            input_ids, attention_mask, token_type_ids, labels = next(source_iterator)
            inputs = {
                "input_ids": input_ids.squeeze(axis=1),
                "attention_mask": attention_mask.squeeze(axis=1),
                "token_type_ids" : token_type_ids.squeeze(axis=1),
                "labels" : labels,
            }

            for k, v in inputs.items():
                inputs[k] = v.to(device)

            attack_pred, pooled_output_prj_source = model(**inputs)
            loss_s_attack = loss_fn_attack_classifier(attack_pred, inputs["labels"])


            # Target dataset training update
            input_ids, attention_mask, token_type_ids, labels = next(target_iterator)
            inputs = {
                "input_ids": input_ids.squeeze(axis=1),
                "attention_mask": attention_mask.squeeze(axis=1),
                "token_type_ids" : token_type_ids.squeeze(axis=1),
                "labels" : labels,
            }

            for k, v in inputs.items():
                inputs[k] = v.to(device)
            _, pooled_output_prj_target = model(**inputs)


            # Combining the loss

            transfer_loss = mkmmd_loss(pooled_output_prj_source, pooled_output_prj_target); ## mk mme loss

#             scf_loss = SmoothCFTest(pooled_output_prj_source,  pooled_output_prj_target, scale = 1, num_random_features=5, method="pinverse" ,device=device)
#             transfer_loss = scf_loss.compute_pvalue() #SCF Loss

#             mkme_loss = MeanEmbeddingTest(
#                   pooled_output_prj_source, pooled_output_prj_target, scale=1, number_of_random_frequencies=5, method="pinverse" ,device=device)
#             transfer_loss = mkme_loss.compute_pvalue()
            loss = loss_s_attack + transfer_loss*1.0
            loss.backward()
            optimizer.step()
            loss_after_epoch = loss
        print(loss_after_epoch)
    evaluate(model, dataset = "transfer", percentage = 100)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Some weights of the model checkpoint at jackaduma/SecBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor(0.2215, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.6654, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.3127, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6070, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.2435, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-0.5028, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.6571, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-0.3010, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.8776, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.5582, device='cuda:0', grad_fn=<AddBackward0>)

 clasification report:
               precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      4580
           1     0.2036    0.0704    0.1046       483
           2     0.5505    0.9566    0.6988      5276
           3     0.0328    1.0000    0.0636        64
           4     0.0000    0.0000    0.0000        22
           5     0.0000    0.0000    0.0000       659
           6     0.6667    0.4844 

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:13

tensor(0.2633, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.4765, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.4665, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0319, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.9201, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0669, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-0.2337, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.1060, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0122, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.2080, device='cuda:0', grad_fn=<AddBackward0>)

 clasification report:
               precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      4081
           1     0.2216    0.0815    0.1192       454
           2     0.6371    0.9659    0.7678      6047
           3     0.0410    0.9412    0.0786        85
           4     0.0000    0.0000    0.0000        30
           5     0.0000    0.0000    0.0000       634
           6     0.3895    0.4262  

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:13

tensor(0.5957, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-0.1251, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-0.6197, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.4185, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.3242, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-0.0382, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-0.0376, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0352, device='cuda:0', grad_fn=<AddBackward0>)
tensor(-0.2311, device='cuda:0', grad_fn=<AddBackward0>)
tensor(0.0013, device='cuda:0', grad_fn=<AddBackward0>)

 clasification report:
               precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      3761
           1     0.2096    0.0785    0.1142       446
           2     0.6449    0.9657    0.7733      6122
           3     0.0282    0.9821    0.0548        56
           4     0.0000    0.0000    0.0000        35
           5     0.0000    0.0000    0.0000       849
           6     0.4738    0.46

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:13

In [18]:
def classifaction_report_csv(report,precision,recall,f1_score,fold):
    """Generate the report to data processing"""
    with open('classification_report_cost.csv', 'a') as f:
        report_data = list()
        lines = report.split('\n')
        row = {}
        row['class'] =  "fold %u" % (fold+1)
        report_data.extend(row)
        print(report_data)
        for line in lines[2:44]:
            row = {}
            line = " ".join(line.split())
            row_data = line.split(' ')
            if(len(row_data)>2):
                if(row_data[0]!='accuracy'):
                    print(row_data)
                    row['class'] = row_data[0]
                    row['precision'] = float(row_data[1])
                    row['recall'] = float(row_data[2])
                    row['f1_score'] = float(row_data[3])
                    row['support'] = row_data[4]
                    report_data.extend(row)
                elif (row_data[1]=='avg'):
                    row['class'] = row_data[0]+row_data[1]
                    row['precision'] = float(row_data[2])
                    row['recall'] = float(row_data[3])
                    row['f1_score'] = float(row_data[4])
                    row['support'] = row_data[5]
                    report_data.extend(row)
                else :
                    row['class'] = row_data[0]+row_data[1]+row_data[2]
                    row['precision'] = float(row_data[3])
                    row['recall'] = float(row_data[4])
                    row['f1_score'] = float(row_data[5])
                    row['support'] = row_data[6]
                    report_data.extend(row)
        row = {}
        row['class'] = 'macro'
        row['precision'] = float(precision)
        row['recall'] = float(recall)
        row['f1_score'] = float(f1_score)
        row['support'] = 0
        report_data.extend(row)
        dataframe = pd.DataFrame.from_dict(report_data)
        dataframe.to_csv(f, index = False)

In [19]:
 torch.save(model.state_dict(), os.path.join(training_parameters["output_folder"], "epoch_" + str(n_epochs)  +  training_parameters["output_file"] ))

In [20]:
evaluate(model, dataset = "transfer", percentage = 100)


 clasification report:
               precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      3761
           1     0.2096    0.0785    0.1142       446
           2     0.6449    0.9657    0.7733      6122
           3     0.0282    0.9821    0.0548        56
           4     0.0000    0.0000    0.0000        35
           5     0.0000    0.0000    0.0000       849
           6     0.4738    0.4600    0.4668       550

    accuracy                         0.5292     11819
   macro avg     0.1938    0.3552    0.2013     11819
weighted avg     0.3641    0.5292    0.4269     11819

F1 score: 0.20130479286831005
Recall: 0.35518795399256037
Precision: 0.19377434228638932
Acc: 0.5292325915898131
Confusion Matrix: 
 [[   0  102 2470 1189    0    0    0]
 [   0   35  393   18    0    0    0]
 [   0   25 5912  185    0    0    0]
 [   0    1    0   55    0    0    0]
 [   0    4    1   30    0    0    0]
 [   0    0  115  453    0    0  281]
 [   0    0  277 

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:13

In [21]:
evaluate(model, dataset = "combine", percentage = 10)


 clasification report:
               precision    recall  f1-score   support

           0     0.9636    0.9494    0.9565     23678
           1     0.9611    0.9886    0.9747     15810
           2     0.9804    0.3861    0.5541      1168
           3     0.6067    0.6113    0.6090       265
           4     0.7830    0.9968    0.8770      1871
           5     0.9943    0.9999    0.9971     16387

    accuracy                         0.9627     59179
   macro avg     0.8815    0.8220    0.8281     59179
weighted avg     0.9645    0.9627    0.9606     59179

F1 score: 0.8280528618176124
Recall: 0.8220266840729789
Precision: 0.8815254232381836
Acc: 0.9627232633197587
Confusion Matrix: 
 [[22479   597     9    93   499     1]
 [  180 15630     0     0     0     0]
 [  661    34   451    12     9     1]
 [    2     0     0   162     9    92]
 [    4     2     0     0  1865     0]
 [    1     0     0     0     0 16386]]
{0: 23678, 1: 15810, 2: 1168, 3: 265, 4: 1871, 5: 16387, 6: 0}


In [22]:
model = DomainAdaptationModel()
model.to(device)
model.load_state_dict(torch.load(os.path.join(training_parameters["output_folder"], "epoch_" + str(n_epochs)  +  training_parameters["output_file"])))

Some weights of the model checkpoint at jackaduma/SecBERT were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>